# Sumarização com algoritimo LUHN

O algoritmo de LUHN
- Seleciona palavras de amiro importancia baseado na frequencia
- Pesos maiores são associados á palavras no inicio do documento
- Para calcular a nota de uma sentença: 4 elevado 2/6=2,7

In [1]:
import re
import nltk
import string
import heapq

In [2]:
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /home/derpy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/derpy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:

def preprocessamento(texto):
    stopwords = nltk.corpus.stopwords.words("portuguese")
    # deixando o texto em minusculo
    texto_formatado = texto.lower()
    tokens = []
    
    # tokenizando o texto
    for token in nltk.word_tokenize(texto_formatado):
        tokens.append(token)
    # removendo as stopwords
    tokens = [palavra for palavra in tokens if palavra not in stopwords]
    
    # removendo a pontuação
    tokens = [palavra for palavra in tokens if palavra not in string.punctuation]
    
    # formatando a lista em string
    texto_formatado = " ".join([str(elemento) for elemento in tokens if not elemento.isdigit()])
    
    return texto_formatado

#### Função para calcular a nota das sentenças

In [4]:
def calcular_nota_sentenca(sentencas,palavras_importantes, distancia):
    notas = []
    indice_sentenca = 0
    
    for sentenca in [nltk.word_tokenize(sentenca.lower()) for sentenca in sentencas]:
        # print("-------")
        # print(sentenca)
        indice_palavra = []
        for palavras in palavras_importantes:
            try:
                indice_palavra. append(sentenca.index(palavras))
            except ValueError:
                pass
        indice_palavra.sort()
        # print(indice_palavra)
        if len(indice_palavra) == 0:
            continue
        lista_grupos = []
        grupo = [indice_palavra[0]]
        i = 1
        while i< len(indice_palavra):
            if indice_palavra[i] - indice_palavra[i - 1] < distancia:
                grupo.append(indice_palavra[i])
                # print("grupo: ", grupo)
            else:
                lista_grupos.append(grupo[:])
                grupo = [indice_palavra[i]]
                # print("grupo: ", grupo)
            i += 1
            lista_grupos.append(grupo)
            # print("todos os grupos: ", lista_grupos)
            nota_maxima_grupo = 0
            
            for g in lista_grupos:
                # print(g)
                palavras_importantes_grupo = len(g)
                total_palavras_grupo = g[-1] - g[0] + 1
                # print("palavras_importantes: " , palavras_importantes_grupo)
                # print("total_palavras: ", total_palavras_grupo)
                nota = 1.0 * palavras_importantes_grupo**2 / total_palavras_grupo
                # print("Nota grupo: ", nota)
                
                if nota > nota_maxima_grupo:
                    nota_maxima_grupo = nota
            notas.append((nota_maxima_grupo, indice_sentenca))
            indice_sentenca+= 1
    #print("notas finais: ", notas)
    return notas

#### Função pra sumarizar os textos

In [5]:
def sumarizar(texto, top_n_palavra, distancia,quantidade_sentencas):
    sentencas_originais = [sentenca for sentenca in nltk.sent_tokenize(texto)]
    sentencas_formatadas = [preprocessamento(sentenca) for sentenca in sentencas_originais]
  
    palavra = [palavra for sentenca in sentencas_formatadas for palavra in nltk.word_tokenize(sentenca)]
    frequencia = nltk.FreqDist(palavra)
 
    top_palavras = [palavra[0] for palavra in frequencia.most_common(top_n_palavra)]
    notas_sentencas = calcular_nota_sentenca(sentencas_formatadas, top_palavras, distancia)
    
    melhores_sentencas = heapq.nlargest(quantidade_sentencas,notas_sentencas)
    melhores_sentencas = [sentencas_originais[i] for (nota, i) in melhores_sentencas]
    return sentencas_originais, melhores_sentencas, notas_sentencas, # top_palavras

In [6]:
texto_original = """A inteligência artificial é a inteligência similar à humana. Definem como
o estudo de agente artificial com inteligência. Ciência e engenharia de
produzir máquinas com inteligência. Resolver problemas e possuir
inteligência. Relacionada ao comportamento inteligente. Construção de
máquinas para raciocinar. Aprender com os erros e acertos. Inteligência
artificial é raciocinar nas situações do cotidiano."""


In [7]:
sentencas_originais, melhores_sentencas, notas_sentencas =  sumarizar(texto_original,5, 3, 3)

In [8]:
sentencas_originais

['A inteligência artificial é a inteligência similar à humana.',
 'Definem como\no estudo de agente artificial com inteligência.',
 'Ciência e engenharia de\nproduzir máquinas com inteligência.',
 'Resolver problemas e possuir\ninteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de\nmáquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência\nartificial é raciocinar nas situações do cotidiano.']

In [9]:
melhores_sentencas

['Aprender com os erros e acertos.',
 'Definem como\no estudo de agente artificial com inteligência.',
 'Construção de\nmáquinas para raciocinar.']

In [10]:
notas_sentencas

[(2.0, 0), (2.25, 1), (2.0, 2), (2.0, 3), (2.0, 4), (2.0, 5), (3.0, 6)]

# Visualização do Resumo

In [11]:
def visualiza_resumo(titulo, lista_sentencas, melhores_sentencas):

    from IPython.core.display import HTML
    texto = ""

    display(HTML(f"<h1>Resumo do texto</h1>"))
    for sentenca in lista_sentencas:
        if sentenca in melhores_sentencas:
            texto += str(sentenca).replace(sentenca, f"<mark>{sentenca}</mark>")
        else:
            texto += sentenca
    display(HTML(f"""{texto}"""))

In [12]:
visualiza_resumo("teste", sentencas_originais, melhores_sentencas)

# Extraindo texto da internet

In [13]:
from goose3 import Goose

In [15]:
g = Goose()
artigo = g.extract(url="https://iaexpert.academy/2021/10/25/ia-atinge-novo-patamar-geracao-imagens-corpo-inteiro-a-partir-uma-unica-foto/")

In [16]:
artigo.cleaned_text

'A geração de imagens humanas fotorrealistas tem aplicações que incluem a criação de avatares virtuais, provadores virtuais, transferência de movimentos e a chamada síntese de posicionamento, onde várias imagens são usadas para gerar uma imagem de um novo ponto de vista. No caso de pessoas, esta tarefa é particularmente complicada para as abordagens tradicionais de processamento gráfico porque ela envolve a geometria tridimensional do corpo e das roupas; o controle das poses que é limitado pela estrutura física dos corpos e que causa deformações na forma; e a síntese de padrões de dobra que ocorrem, por exemplo, com roupas largas. Recentemente, métodos baseados em inteligência artificial mostraram resultados promissores utilizando redes generativas adversariais, mas para aplicações muito particulares, não atingindo o grau de generalização necessário para seu uso indiscriminado.\n\nPartindo destes métodos, pesquisadores da Virginia Tech, nos Estados Unidos, em parceria com a Adobe, apre

In [24]:
sentencas_originais, melhores_sentencas, notas_sentencas =  sumarizar(artigo.cleaned_text,10, 10, 3)

In [25]:
sentencas_originais

['A geração de imagens humanas fotorrealistas tem aplicações que incluem a criação de avatares virtuais, provadores virtuais, transferência de movimentos e a chamada síntese de posicionamento, onde várias imagens são usadas para gerar uma imagem de um novo ponto de vista.',
 'No caso de pessoas, esta tarefa é particularmente complicada para as abordagens tradicionais de processamento gráfico porque ela envolve a geometria tridimensional do corpo e das roupas; o controle das poses que é limitado pela estrutura física dos corpos e que causa deformações na forma; e a síntese de padrões de dobra que ocorrem, por exemplo, com roupas largas.',
 'Recentemente, métodos baseados em inteligência artificial mostraram resultados promissores utilizando redes generativas adversariais, mas para aplicações muito particulares, não atingindo o grau de generalização necessário para seu uso indiscriminado.',
 'Partindo destes métodos, pesquisadores da Virginia Tech, nos Estados Unidos, em parceria com a A

In [26]:
melhores_sentencas

['Recentemente, métodos baseados em inteligência artificial mostraram resultados promissores utilizando redes generativas adversariais, mas para aplicações muito particulares, não atingindo o grau de generalização necessário para seu uso indiscriminado.',
 'Os detalhes da imagem original são mantidos modulando a rede geradora através técnicas computacionais baseadas em superfície, que garantem a correta transferência da informação de um pixel dadas as restrições impostas pela nova posição.',
 'O modelo usa a rede generativa chamada de StyleGAN, para transferência de estilo, mas numa abordagem condicionada pela pose.']

In [27]:
notas_sentencas

[(2.0, 0),
 (3.0, 1),
 (4.0, 2),
 (3.125, 3),
 (3.272727272727273, 4),
 (3.272727272727273, 5),
 (2.0, 6),
 (1.0, 7),
 (1.6, 8),
 (2.0, 9),
 (2.0, 10),
 (2.0, 11),
 (0.6666666666666666, 12),
 (2.0, 13),
 (3.0, 14)]

In [28]:
visualiza_resumo("teste", sentencas_originais, melhores_sentencas)